<a href="https://colab.research.google.com/github/Michael-HK/Mobile_Apps_Reccomendation_system/blob/main/mobile_Apps_reccomender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import Dependencies**

In [1]:
!pip install contractions
!pip install inflect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 6.0 MB/s eta 0:00:00


## Import libraries

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import gzip
import ast
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import warnings
#%load_ext watermark
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate, Flatten,Dropout, BatchNormalization
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [4]:
from google.colab import drive
drive.mount('/content/drive')

# change directory to the folder
%cd /content/drive/My Drive/end_2_end_ML/music_recommendation

Mounted at /content/drive
/content/drive/My Drive/end_2_end_ML/music_recommendation


## Data Ingest

In [5]:
#======== download datasets ================

!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Apps_for_Android.json.gz

!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Apps_for_Android.json.gz


--2023-07-22 06:38:08--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Apps_for_Android.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299986290 (286M) [application/x-gzip]
Saving to: ‘reviews_Apps_for_Android.json.gz.1’

reviews_Apps_for_An 100%[===================>] 286.09M  1.13MB/s    in 2m 48s  

2023-07-22 06:40:58 (1.70 MB/s) - ‘reviews_Apps_for_Android.json.gz.1’ saved [299986290/299986290]

--2023-07-22 06:40:58--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Apps_for_Android.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16414723 (16M) [application/x-gzip]
Saving to: ‘meta_Apps_for_Android.json.gz.1’

meta_

In [6]:
##======= function to read the file as dataframe =============================

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df1 = getDF('./reviews_Apps_for_Android.json.gz.1')
df2 = getDF('./meta_Apps_for_Android.json.gz.1')

In [8]:
#=== view

#df1.head()
df2.head()

,asin,related,categories,salesRank,price
0,B004A9SDD8,"{'also_bought': ['B006M3K874', 'B00F85SMOI', '...","[[Apps for Android, Kids]]",NaN,NaN
1,B004AFQAUA,"{'also_bought': ['B005UDD1R6', 'B00LBH2UMM', '...","[[Apps for Android, Music, Music Players]]",NaN,NaN
2,B004AGCR1K,"{'also_viewed': ['B00A7W29BE', 'B00I5PB9UM', '...","[[Apps for Android, Productivity]]",NaN,NaN
3,B004AHBBPW,"{'also_bought': ['B006T2U1V6', 'B00FBCIYZU', '...","[[Apps for Android, Reference]]",NaN,NaN
4,B004ALFHV2,"{'also_bought': ['B00CA6LZAG', 'B00FE5ZF4W', '...","[[Apps for Android, Communication]]",NaN,NaN


In [9]:
print(df1.shape)
print(df2.shape)

(2638173, 9)
(61551, 5)


In [10]:
##======== number of products===========

df1['asin'].nunique()

#print('Total number of review Apps is: ', len(df1['asin'].nunique()))

61275

In [11]:
##========== merge the files===============

merged_dataset = pd.merge(df1, df2, on='asin', how='left')

In [12]:
merged_dataset.head(10)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,related,categories,salesRank,price
0,AUI0OLXAB3KKT,B004A9SDD8,A Customer,"[0, 0]",Glad to finally see this app on the android ma...,5.0,Great app!!!,1.301184e+09,"03 27, 2011","{'also_bought': ['B006M3K874', 'B00F85SMOI', '...","[[Apps for Android, Kids]]",NaN,NaN
1,A1ZUSQ3TC3EC4C,B004A9SDD8,A. Lissak,"[12, 14]",this app works great on the Kindle Fire... kid...,5.0,Kid loves it,1.321574e+09,"11 18, 2011","{'also_bought': ['B006M3K874', 'B00F85SMOI', '...","[[Apps for Android, Kids]]",NaN,NaN
2,AC05OAXD72X1V,B004A9SDD8,Allie,"[0, 0]",We love these monkey's and all the concepts th...,4.0,Love these monkeys!,1.367366e+09,"05 1, 2013","{'also_bought': ['B006M3K874', 'B00F85SMOI', '...","[[Apps for Android, Kids]]",NaN,NaN
3,A2RVMFOKBVM21I,B004A9SDD8,Amazon Customer,"[0, 2]",cannot get my kindle away from my 2 year old g...,5.0,fun fun for toddlers,1.350173e+09,"10 14, 2012","{'also_bought': ['B006M3K874', 'B00F85SMOI', '...","[[Apps for Android, Kids]]",NaN,NaN
4,A3NBSRGUWQGCMZ,B004A9SDD8,Amazon Customer,"[1, 3]",I start this app up whenever I forget what a f...,1.0,Might be great if it worked,1.300838e+09,"03 23, 2011","{'also_bought': ['B006M3K874', 'B00F85SMOI', '...","[[Apps for Android, Kids]]",NaN,NaN
5,A2KTVCVZJ8GPD2,B004A9SDD8,A. Mclean,"[0, 0]",Very engaging to my three year old toddler. Sh...,5.0,Great video,1.393546e+09,"02 28, 2014","{'also_bought': ['B006M3K874', 'B00F85SMOI', '...","[[Apps for Android, Kids]]",NaN,NaN
6,A2I9RREBHMMPCJ,B004A9SDD8,AngelaM,"[1, 2]","My daughter loves this app. Unfortunately, af...",3.0,great when it worked,1.305504e+09,"05 16, 2011","{'also_bought': ['B006M3K874', 'B00F85SMOI', '...","[[Apps for Android, Kids]]",NaN,NaN
7,A1N4O8VOJZTDVB,B004A9SDD8,Annette Yancey,"[1, 1]","Loves the song, so he really couldn't wait to ...",3.0,Really cute,1.383350e+09,"11 2, 2013","{'also_bought': ['B006M3K874', 'B00F85SMOI', '...","[[Apps for Android, Kids]]",NaN,NaN
8,A2HQWU6HUKIEC7,B004A9SDD8,"Audiobook lover ""Kathy""","[0, 0]","Oh, how my little grandson loves this app. He'...",5.0,2-year-old loves it,1.323043e+09,"12 5, 2011","{'also_bought': ['B006M3K874', 'B00F85SMOI', '...","[[Apps for Android, Kids]]",NaN,NaN
9,A1SXASF6GYG96I,B004A9SDD8,Barbara Gibbs,"[0, 0]",I found this at a perfect time since my daught...,5.0,Fun game,1.337558e+09,"05 21, 2012","{'also_bought': ['B006M3K874', 'B00F85SMOI', '...","[[Apps for Android, Kids]]",NaN,NaN


In [13]:
# check for null
merged_dataset.isnull().sum()


reviewerID              0
asin                    0
reviewerName       187320
helpful                 0
reviewText              0
overall                 0
summary                 0
unixReviewTime          1
reviewTime              0
related              2768
categories              0
salesRank         2635549
price             2617728
dtype: int64

In [14]:
#=== obtain the main columns that are needed for our system ====================

apps = merged_dataset[['reviewerID', 'asin', 'reviewText','summary', 'categories','overall']]
apps.head()

,reviewerID,asin,reviewText,summary,categories,overall
0,AUI0OLXAB3KKT,B004A9SDD8,Glad to finally see this app on the android ma...,Great app!!!,"[[Apps for Android, Kids]]",5.0
1,A1ZUSQ3TC3EC4C,B004A9SDD8,this app works great on the Kindle Fire... kid...,Kid loves it,"[[Apps for Android, Kids]]",5.0
2,AC05OAXD72X1V,B004A9SDD8,We love these monkey's and all the concepts th...,Love these monkeys!,"[[Apps for Android, Kids]]",4.0
3,A2RVMFOKBVM21I,B004A9SDD8,cannot get my kindle away from my 2 year old g...,fun fun for toddlers,"[[Apps for Android, Kids]]",5.0
4,A3NBSRGUWQGCMZ,B004A9SDD8,I start this app up whenever I forget what a f...,Might be great if it worked,"[[Apps for Android, Kids]]",1.0


In [15]:
##==== check for nan

apps.isnull().sum()


reviewerID    0
asin          0
reviewText    0
summary       0
categories    0
overall       0
dtype: int64

In [16]:
apps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2638173 entries, 0 to 2638172
Data columns (total 6 columns):
 #   Column      Dtype  
---  ------      -----  
 0   reviewerID  object 
 1   asin        object 
 2   reviewText  object 
 3   summary     object 
 4   categories  object 
 5   overall     float64
dtypes: float64(1), object(5)
memory usage: 140.9+ MB


In [17]:
print(apps.shape)

(2638173, 6)


In [19]:
print('numer of users/reviewers:')
print(apps['reviewerID'].nunique())
print('number of android apps:')
print(apps['asin'].nunique())

numer of users/reviewers:
1323884
number of android apps:
61275


## Text preprocessing

In [22]:
#=== text data ============

apps['tags'] = apps['reviewText'] + apps['summary'] + str(apps['categories'])


### Text cleaning and curating

In [36]:
"""Text preprocessing"""

import contractions
import inflect
from nltk.corpus import stopwords
import re
nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text):
    # Lowercase
    text = text.lower()

    # Remove special characters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    text = re.sub(r'(@|#)\w+' , '' , text)
    text = re.sub("[,.]", "", text)
    text = re.sub(r'https?://\S+' , '' , text)
    text = re.sub(r'(\?|!)+' , '' , text)
    text = re.sub(r"\(|\)", "", text)
    text = re.sub(r'(^\s+)' , '' , text)
    text = re.sub(r'(\s+$)' , '' , text)
    text = re.sub(r'\d+' , '' , text)

    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    tokens = text.split()
    tokens = [token for token in tokens if token not in stop_words]
    text = " ".join(tokens)

    # Perform stemming  ps = PorterStemmer()
    stemmer = PorterStemmer()  #"english"
    tokens = text.split()
    tokens = [stemmer.stem(token) for token in tokens]               # stemming
    text = " ".join(tokens)

    # Expand contractions
    tokens  = text.split()
    tokens = [contractions.fix(token) for token in tokens]           # expand words
    text = " ".join(tokens)

    # Convert numeric values to text expressions
    p = inflect.engine()
    tokens = text.split()
    tokens = [p.number_to_words(token) if token.isdigit() else token for token in tokens]
    text = " ".join(tokens)

    return text

def preprocess_data(data):
    if isinstance(data, str):
        # Preprocess a single text
        preprocessed_text = preprocess_text(data)
        return preprocessed_text

    elif isinstance(data, (list, np.ndarray)):
        # Preprocess an array or vector column
        preprocessed_data = [preprocess_text(text) for text in data]
        return np.array(preprocessed_data)

    elif isinstance(data, pd.Series):
        # Preprocess a Series (column)
        preprocessed_data = data.apply(preprocess_text)
        return preprocessed_data

    elif isinstance(data, pd.DataFrame):
        # Preprocess text columns in a DataFrame
        preprocessed_data = data.copy()
        for column in data.columns:
            preprocessed_data[column] = data[column].apply(preprocess_text)
        return preprocessed_data
    else:
        raise ValueError("Input data must be a string, column series, array, or a DataFrame.")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Text tokenization and padding

In [ ]:
"""Deep learning model for Reccomender system"""

# Encode categorical features
id_encoder = LabelEncoder()
title_encoder = LabelEncoder()
apps['reviewerID'] = id_encoder.fit_transform(apps['reviewerID'])
apps['asin'] = title_encoder.fit_transform(apps['asin'])

##========= pickle encoder function for later use ==============================
with open('id_encoder.pkl', 'wb') as id_enc:
  pickle.dump(id_encoder, id_enc)

with open('title_encoder.pkl', 'wb') as title_enc:
  pickle.dump(title_encoder, title_enc)

##========================clean data============================================
apps['tags'] = preprocess_data(apps['tags'])


#====pickle cleaned data+++
Apps_df = apps[['reviewerID', 'asin', 'tags']]


#============== Tokenize and pad apps descriptions=============================
tokenizer = Tokenizer()
tokenizer.fit_on_texts(apps['tags'])
apps['seqs'] = tokenizer.texts_to_sequences(apps['tags'])

#================Split the data into train and test sets========================
train_data, test_data = train_test_split(apps, test_size=0.2, random_state=42)

#======================pad sequence=============================================
max_sequence_length = max(len(seq) for seq in apps['seqs'])
print('maximum sequence length is:', max_sequence_length)
train_padded_sequences = pad_sequences(train_data['seqs'], maxlen=max_sequence_length)
test_padded_sequences = pad_sequences(test_data['seqs'], maxlen=max_sequence_length)

## Building Model

### Wide_deep_model

In [ ]:
############### Building Model ======================

def wide_deep_model(apps, max_sequence_length):
    """Define input layers"""

    reviewerID = Input(shape=(1,), name='reviewerID')
    asin = Input(shape=(1,), name='asin')
    tags = Input(shape=(max_sequence_length,), name='tags')

    # Embedding layers for reviewerID and asin
    id_embedding = Embedding(input_dim=apps['reviewerID'].nunique(), output_dim=10, input_shape=(1,))(reviewerID)
    title_embedding = Embedding(input_dim=apps['asin'].nunique(), output_dim=10, input_shape=(1,))(asin)

    # Flatten the embedding layers
    id_flatten = Flatten()(id_embedding)
    title_flatten = Flatten()(title_embedding)

    # Concatenate the flattened embeddings
    concatenated = Concatenate()([id_flatten, title_flatten])

    # Deep component
    bn_concat = BatchNormalization()(concatenated)
    fc1 = Dense(256, use_bias=False, activation='relu')(bn_concat)
    bn1 = BatchNormalization()(fc1)
    fc1 = Dropout(rate=0.2)(bn1)

    fc2 = Dense(128, use_bias=False, activation='relu')(fc1)
    bn2 = BatchNormalization()(fc2)
    fc2 = Dropout(rate=0.2)(bn2)

    fc3 = Dense(64, use_bias=False, activation='relu')(fc2)
    bn3 = BatchNormalization()(fc3)
    fc3 = Dropout(rate=0.2)(bn3)

    deep_output = Dense(64, activation='relu')(fc3)

    # Fully connected layers for the wide component
    wide_output = Concatenate()([tags, deep_output])
    wide_output = Dropout(rate=0.2)(wide_output)
    wide_output = Dense(1)(wide_output)

    # Model
    model = Model(inputs=[reviewerID, asin, tags], outputs=wide_output)

    # Compile the model
    model.compile(optimizer='adam', loss='mse', metrics=['mse'])
    return model


#### Train wide-deep-model

In [ ]:
model = wide_deep_model(apps, max_sequence_length)

# Train the model
#model.fit(x = {'reviewerID':reviewerID,
#               'asin': asin,
#               'tags': train_padded_sequences},
#          y = train_data['overall'],
#          epochs=100, batch_size=32, validation_split=0.2)

# Assuming you have processed the data and created padded_sequences
train_reviewerID = train_data['reviewerID'].values
train_asin = train_data['asin'].values
train_tags = train_padded_sequences

# Train the model
model.fit(
    x=[train_reviewerID, train_asin, train_tags],
    y=train_data['overall'],
    epochs=100,
    batch_size=64,
    validation_split=0.2
)

### Evaluate trained Model

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# test_data
test_reviewerID = test_data['reviewerID'].values
test_asin = test_data['asin'].values
test_tags = test_padded_sequences

# Evaluate the model on the test data
predictions = model.predict([test_reviewerID, test_asin, test_tags]).flatten()

# Replace 'target_column' with the actual target column in your dataset
true_values = test_data['overall'].values

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(true_values, predictions)

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(true_values, predictions)

# Calculate R-squared (R2) score
r2 = r2_score(true_values, predictions)

print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2) score:", r2)


In [ ]:
import numpy as np

def Apps_recommendations(model, data, user_id, k=5):
    """ Apps reccomendation based on users record"""

    # Create sequences of apps descriptions for the user
    user_apps_reviews = data.loc[data['reviewerID'] == user_id, 'tags']             ## data need to be changed "tags":apps reviews
    user_sequences = tokenizer.texts_to_sequences(user_apps_reviews)
    user_padded_sequences = pad_sequences(user_sequences, maxlen=max_sequence_length)

    # Prepare input data for the model
    user_input = np.full_like(user_padded_sequences, user_id)                       #to ensure userID as the same numbers with other input
    asin_input = data.loc[data['reviewerID'] == user_id, 'asin'].values             #encoded apps ID

    # Get apps ID and description indices with highest predicted ratings
    predictions = model.predict([user_input, asin_input, user_padded_sequences]).flatten()  # input: userID, AppsID, tokenised & vectorised apps reviews
    top_k_indices = np.argsort(predictions)[::-1][:k]

    # Get the andriod Apps recommendation for the top-k recommendations
    top_k_asins = data.loc[top_k_indices, 'asin'].values

    asin_encoder = joblib.load('id_encoder.pkl')

    top_k_AppsID = asin_encoder.inverse_transform(top_k_asins)

    #top_k_movie_titles = data.loc[data['asin'].isin(top_k_asins), 'movie_title'].values

    return top_k_AppsID

# Example usage to get movie recommendations for user with ID 123
#user_id = 123
#recommendations = Apps_recommendations(model, user_id)
#print(f"Apps recommendations for user {user_id}:")
#print(recommendations)


**Evaluate trained Reccommendation model**

In [ ]:
# Reccomendation Model evaluation

def precision_recall_at_k(model, test_data, k=5):
    precision_sum = 0
    recall_sum = 0

    for user_id in test_data['reviewerID'].unique():
        user_apps = test_data[test_data['reviewerID'] == user_id]['asin'].values
        user_predictions = Apps_recommendations(model, user_id, k=k)

        # Calculate intersection (retrieved items that are relevant)
        intersection = len(set(user_apps) & set(user_predictions))

        # Calculate precision and recall for this user
        precision = intersection / min(k, len(user_apps))
        recall = intersection / len(user_apps)

        precision_sum += precision
        recall_sum += recall

    # Calculate average precision and recall across all users
    avg_precision = precision_sum / len(test_data['reviewerID'].unique())
    avg_recall = recall_sum / len(test_data['reviewerID'].unique())

    return avg_precision, avg_recall


In [ ]:
# Reccomendation Model evaluation function

k_values = [5, 10, 15, 20]

for k in k_values:
  pre, re = precision_recall_at_k(model, test_data, k=k)
  print('Proposed Reccomendation system Evaluation')
  print(f'The average precision and average recall at {k} are {pre} and {re}, respectively')

## Recommendation based on likes or view

In [ ]:
## The input into cv i.e apps['tags'] must be a clean text data

cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(apps['tags']).toarray()

#vectors
similarity = cosine_similarity(vectors)    # for similarity evaluation

In [ ]:
##======= save model ===================
pickle.dump(Apps_df.to_dict(), open('apps_dict.pkl', 'wb'))   #save_cleaned data
pickle.dump(similarity, open('similarity.pkl', 'wb'))

pickle.dump(model, open('Recommend_model.pkl', 'wb'))

In [ ]:
#=========== Recommendation system =============================================

def app_recommend(Apps_df, similarity, appID):
  """load save model"""
 # Apps_df = joblib.load('apps_dict.pkl')
 # similarity = joblib.load('similarity.pkl')

  apps_index = Apps_df[Apps_df['asin'] == appID].index[0]
  distances = similarity[apps_index]
  apps_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]

  for i in apps_list:
    print(Apps_df.iloc[i[0]].asin)

## Integrated Reccomendation Model based on User Records and current view/likes

In [ ]:
import joblib

class Apps_Reccomendation_Model():
  def __init__(self):
    self.data = joblib.load('data_df.pkl')
    self.model = joblib.load('Recommend_model.pkl')
    self.Apps_df = joblib.load('apps_dict.pkl')
    self.similarity = joblib.load('similarity.pkl')

  def integral_recommend(self, userID: None, appID:None):
    if userID is not None and appID is None:
      print('Reccommended apps based on your previous download')
      reccomend = Apps_recommendations(self.model, userID, k=5)
      print(reccomend)

    elif userID is None and appID is not None:
      print('Reccommended items based on current view/likes:')
      app_recommend(self.Apps_df, self.similarity, appID)

    elif userID is not None and appID is not None:
      print('Reccommended items based on current view/likes:')
      app_recommend(self.Apps_df, self.similarity, appID)

      print('Reccommended apps based on your previous download')
      reccomend = Apps_recommendations(self.model, userID, k=5)
      print(reccomend)
    else:
      print('provide your ID or view one of the apps')